In [1]:
from datalab.context import Context
import google.datalab.storage as storage
import google.datalab.bigquery as bq
import pandas as pd
import os
from google.cloud import bigquery

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="auth/dados-publicos-326316-e16540376290.json"

In [6]:
# # Imports the Google Cloud client library
# from google.cloud import storage

# # Instantiates a client
# storage_client = storage.Client()

# # The name for the new bucket
# bucket_name = "dna_public_teste"

# # Creates the new bucket
# bucket = storage_client.create_bucket(bucket_name)

# print("Bucket {} created.".format(bucket.name))

Bucket dna_public_teste created.


In [7]:
client = bigquery.Client()
dataset_id = f"{client.project}.dna_dataset"

# Criar um dataset novo no projeto

In [6]:
# Construct a BigQuery client object.
client = bigquery.Client()

# TODO(developer): Set dataset_id to the ID of the dataset to create.
dataset_id = f"{client.project}.dna_dataset"

# Construct a full Dataset object to send to the API.
dataset = bigquery.Dataset(dataset_id)

# TODO(developer): Specify the geographic location where the dataset should reside.
dataset.location = "US"

# Send the dataset to the API for creation, with an explicit timeout.
# Raises google.api_core.exceptions.Conflict if the Dataset already
# exists within the project.
dataset = client.create_dataset(dataset, timeout=30)  # Make an API request.
print(f"Created dataset {client.project}.{dataset.dataset_id}")

Created dataset dados-publicos-326316.dna_dataset


# Criando tabelas com os schemas

In [8]:
# Set table_id to the ID of the table to create.
table_id = f"{dataset_id}.simples_format"

schema = [
    bigquery.SchemaField("cnpj", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("opcao_simples", "STRING"),
    bigquery.SchemaField("data_opcao_simples", "DATE", mode="NULLABLE"),
    bigquery.SchemaField("data_exclusao_simples", "DATE", mode="NULLABLE"),
    bigquery.SchemaField("opcao_mei", "STRING"),
    bigquery.SchemaField("data_opcao_mei", "DATE", mode="NULLABLE"),
    bigquery.SchemaField("data_exclusao_mei", "DATE", mode="NULLABLE"),
]

table = bigquery.Table(table_id, schema=schema)
table = client.create_table(table)  # Make an API request.
print(
    "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
)

Created table dados-publicos-326316.dna_dataset.simples_format


# Salvando do GCS para o BigQuery

In [25]:
# Construct a BigQuery client object.
client = bigquery.Client()

directory = 'empresas'
dataset_id= f"{client.project}.dna_test_dataset"

# Set table_id to the ID of the table to create.
table_id = f"{dataset_id}.empresas_teste"

job_config = bigquery.LoadJobConfig(source_format=bigquery.SourceFormat.PARQUET,)
uri = f"gs://dna_public_cnpj_files/empresas_trusted_parquet/*.parquet"

load_job = client.load_table_from_uri(
    uri, table_id, job_config=job_config
)  # Make an API request.

load_job.result()  # Waits for the job to complete.

destination_table = client.get_table(table_id)
print("Loaded {} rows.".format(destination_table.num_rows))

Loaded 46906616 rows.
